In [37]:
import pandas as pd
import numpy as np

In [38]:
df= pd.read_csv('C:/Users/prathyusha/OneDrive/Documents/Datasets/pima-indians-diabetes.csv',header=None,names=features)
features = ['preg','plas','pres','skin','test','mass','pedi','age','Target']
df

,preg,plas,pres,skin,test,mass,pedi,age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [39]:
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
 df.shape

(768, 9)

In [72]:
df.isnull().sum()

preg      0
plas      0
pres      0
skin      0
test      0
mass      0
pedi      0
age       0
Target    0
dtype: int64

# Preparing the data

In [41]:
data = df.values
X = data[:,0:8]
Y = data[:,8]

# FILTER METHOD

In [29]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [31]:
#Feature extraction
chi_best = SelectKBest(score_func=chi2,k=4)
k_best = chi_best.fit(X,Y)
#summarize scores
np.set_printoptions(precision=3)
print(k_best.scores_)
k_features = k_best.transform(X)
#Summarize selected features
print(k_features[0:5,:])

[ 111.52  1411.887   17.605   53.108 2175.565  127.669    5.393  181.304]
[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 [ 89.   94.   28.1  21. ]
 [137.  168.   43.1  33. ]]


# Wrapper Method

Recursive Feature Elimination(RFE)

In [42]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [55]:
#Feature Extraction
model_lr = LogisticRegression()
selector = RFE(model_lr,n_features_to_select=3,step=1)
selector = selector.fit(X, Y)
print("Number of Features: %s" % (selector.n_features_to_select))
print("Selected Features are:%s" % (selector.support_))
print("Feature Ranking is as follows: %s" % (selector.ranking_))

Number of Features: 3
Selected Features are:[ True False False False False  True  True False]
Feature Ranking is as follows: [1 2 4 5 6 1 1 3]


# EMBEDDED METHOD

L2 Regularization

In [56]:
from sklearn.linear_model import Ridge

In [58]:
ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X,Y)

Ridge()

In [59]:
#A helper function for printing the coefficients
def print_coefs(coef,names=None,sort=False):
    if names == None:
        names = ["X%s" % x for x in range(len(coef))]
    lst = zip(coef,names)
    if sort:
        lst = sorted(lst, key = lambda x:-np.abs(x[0]))
    return "+".join("%s * %s" % (round(coefs,3),name)
                                 for coefs, name in lst)

In [61]:
print("Ridge model:",print_coefs(ridge_reg.coef_))

Ridge model: 0.021 * X0+0.006 * X1+-0.002 * X2+0.0 * X3+-0.0 * X4+0.013 * X5+0.145 * X6+0.003 * X7


In [64]:
X = df[['preg','pedi','age']]
Y = df[['Target']]

In [66]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=99, test_size=0.3)

In [67]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

In [68]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression()

In [69]:
y_pred = classifier.predict(x_test)

In [70]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
 
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[131  24]
 [ 50  26]]


In [71]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.6796536796536796
